# 다중 선형 회귀(Multiple Linear Regression) : y = m1x1 + m2x2 + m3x3 ... + b
원핫 인코딩 : 회귀 모델은 연속형 데이터만 처리, 분류형 데이터의 경우 수정이 필요한데 사용할 수 있는 방법이 원핫 인코딩 => 더미컬럼

다중 공선성 : 독립 변수들 간에 서로 강한 상관관계를 가지면서 회귀계수 추정의 오류가 나타나는 문제(하나의 피처가 다른 피처에 영향을 끼침)
=> 어느정도의 상관관계는 괜찮지만 높은 상관관계의 경우 처리를 해줘야함
=> 원핫 인코딩의 경우 하나의 컬럼을 없애는 방법이 있음
원래의 원핫 인코딩은 D1 + D2 + D3 = 1(세 개의 컬럼중 어쨋든 다 더하면 1이 나오게 되어있음)이므로 D3 = 1 - (D1 + D2) 라는 강한 상관관계 -> 하나의 컬럼을 없앰으로써 이 식을 무너뜨림
-> Dummy variable trap

### 원 핫 인코딩

In [1]:
import pandas as pd

In [2]:
dataset = pd.read_csv('./MultipleLinearRegressionData.csv')

In [6]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values
X

array([[0.5, 3, 'Home'],
       [1.2, 4, 'Library'],
       [1.8, 2, 'Cafe'],
       [2.4, 0, 'Cafe'],
       [2.6, 2, 'Home'],
       [3.2, 0, 'Home'],
       [3.9, 0, 'Library'],
       [4.4, 0, 'Library'],
       [4.5, 5, 'Home'],
       [5.0, 1, 'Cafe'],
       [5.3, 2, 'Cafe'],
       [5.8, 0, 'Cafe'],
       [6.0, 3, 'Library'],
       [6.1, 1, 'Cafe'],
       [6.2, 1, 'Library'],
       [6.9, 4, 'Home'],
       [7.2, 2, 'Cafe'],
       [8.4, 1, 'Home'],
       [8.6, 1, 'Library'],
       [10.0, 0, 'Library']], dtype=object)

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(drop='first'), [2])], remainder = 'passthrough')
X = ct.fit_transform(X)
X

# 1 0 : Home
# 0 1 : Library
# 0 0 : Cafe

array([[1.0, 0.0, 0.5, 3],
       [0.0, 1.0, 1.2, 4],
       [0.0, 0.0, 1.8, 2],
       [0.0, 0.0, 2.4, 0],
       [1.0, 0.0, 2.6, 2],
       [1.0, 0.0, 3.2, 0],
       [0.0, 1.0, 3.9, 0],
       [0.0, 1.0, 4.4, 0],
       [1.0, 0.0, 4.5, 5],
       [0.0, 0.0, 5.0, 1],
       [0.0, 0.0, 5.3, 2],
       [0.0, 0.0, 5.8, 0],
       [0.0, 1.0, 6.0, 3],
       [0.0, 0.0, 6.1, 1],
       [0.0, 1.0, 6.2, 1],
       [1.0, 0.0, 6.9, 4],
       [0.0, 0.0, 7.2, 2],
       [1.0, 0.0, 8.4, 1],
       [0.0, 1.0, 8.6, 1],
       [0.0, 1.0, 10.0, 0]], dtype=object)

### 데이터 세트 분리

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2) # 20%의 테스트 세트

### 학습 (다중 선형 회귀)

In [9]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(X_train, y_train)

LinearRegression()

### 예측 값과 실제 값 비교 (테스트 세트)

In [10]:
y_pred = reg.predict(X_test)
y_pred, y_test

(array([48.67357773, 89.53350441, 55.28261687, 75.91305223]),
 array([48, 90, 58, 76]))

In [12]:
reg.coef_ # 독립 변수 4개 집 공부와 도서관 공부는 점수에 좋지 않은 영향, 공부 시간은 점수에 긍정적인 영향, 결석횟수는 점수에 부정적인 역할

array([-5.60835791, -2.16219683, 10.11474566, -1.62200508])

In [13]:
reg.intercept_

6.330893673259695

### 모델 평가
모델이 좋은지, 신뢰할 수 있는지 파악 => 개선할 점 찾기, 성능 비교로 더 나은 모델 선택

In [14]:
reg.score(X_train, y_train), reg.score(X_test, y_test)

(0.9702403718774308, 0.99230624420063)

#### 회귀 모델의 모델 평가
MAE => 실제 값과 예측 값 차이의 절대값 평균
MSE => 실제 값과 예측 값 차이의 제곱 평균(이상값에 민감하다는 단점, 단위를벗 어나 버릴 수 있는 문제)
RMSE => MSE에 루트(1보다 작은 값은 너무 작아지고, 1보다 큰 값은 너무 커지는 문제 상쇄, 실제 단위와 통일)
>0에 가까울 수록 좋은 모델임

R-Square(R2) : 결정계수
1 - SSE/SST = 1 - (실제 값 - 예측 값)^2 / (실제 값 - 평균 값)^2
예측 값과 평균 값이 비슷하다면 1 - 1 = 0 값이 됨
에측 값이 실제 값과 비슷하다면 1 - 0 = 1 값이 됨
> 1에 가까울수록 좋고, 0에 가까울 수록 좋지 않음

error(잔차) : 실제 값 - 예측 값
reg : 예측 값 - 평균 값
total : 실제 값 - 평균 값 = error + reg

잔차의 제곱 합 : Sum Square error = SSE
reg의 제곱 합 : Sum Square reg = SSR
total의 제곱 합 : Sum Suare total = SST

#### 다양한 평가 지표(회귀 모델)

In [15]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, y_pred) # 실제값, 예측값의 MAE

0.9861010546626527

In [17]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_pred) # 실제값, 예측값의 MSE

2.0157640194349487

In [18]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_pred, squared=False) # 실제값, 예측값의 RMSE

1.4197760455208943

In [21]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred) # 실제값, 예측값의 R2

0.99230624420063

Regression의 모델 평가 점수는 R2 점수로 매개진다는 것을 확인할 수 있음

# 다항 선형 회귀(Polynomial Linear Regression) : y = m1x^1 + m2x^2 + m3x^3 ... + b
원래 있던 피쳐를 확장하는 개념 -> 데이터를 더 잘 표현하기 위함(ex) 1차 직선 -> 2차 곡선)
구글 검색 : polynomial regression data fit => 해당 사이트에서 비쥬얼 적으로 그래프 조작하면서 확인 가능

과소적합 : 데이터를 잘 표현하지 못하는 상황 => 개선 여지
과대적합 : 훈련 데이터는 매우 잘 표현하지만, 모델이 너무 훈련 데이터에만 적합하여 테스트 데이터 및 실제 데이터는 잘 표현하지 못합